In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorboardX import SummaryWriter
import time
import numpy as np

In [2]:
# Configuration Variables
MEAN_PREPRO = [0.485, 0.456, 0.406]
STD_PREPRO = [0.229, 0.224, 0.225]
RESIZE_PREPRO = 256,256

TRAIN_BATCH_SIZE = 128
TRAIN_SHUFFLE = True
TRAIN_NUM_WORKERS = 8
TRAIN_PIN_MEMORY = True

VAL_BATCH_SIZE = 512
VAL_SHUFFLE = False
VAL_NUM_WORKERS = 4
VAL_PIN_MEMORY = True

INITIAL_LR = 1e-4
DEVICE_ID = 1

DATA_DIR = '/data/porn/data'
TRAINSET_ROOT = f'{DATA_DIR}/train'
TESTSET_ROOT = f'{DATA_DIR}/test'

TENSORBOARD_DIR = '/data/porn/tensorboard/resnet50/'

In [3]:
normalize = transforms.Normalize(
        mean=MEAN_PREPRO, std=STD_PREPRO
    )
prepro = transforms.Compose(
    [
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]
)
prepro_val = transforms.Compose(
    [transforms.Resize(RESIZE_PREPRO), transforms.ToTensor(), normalize]
)

In [4]:
train_dataset = ImageFolder(TRAINSET_ROOT, transform=prepro)
val_dataset = ImageFolder(TESTSET_ROOT, transform=prepro_val)

In [5]:
print(train_dataset)
print(train_dataset.classes)

Dataset ImageFolder
    Number of datapoints: 183019
    Root location: /data/porn/data/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(256, 256), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['drawings', 'hentai', 'neutral', 'porn']


In [6]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=VAL_SHUFFLE, num_workers=VAL_NUM_WORKERS, pin_memory=VAL_PIN_MEMORY, drop_last=False)

In [7]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(2048, len(train_dataset.classes))
DEVICE = f"cuda:{DEVICE_ID}"
model = model.to(DEVICE)
for p in model.parameters():
    p.requires_grad=False
for p in model.fc.parameters():
    p.requires_grad=True

In [8]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR)
criterion = nn.CrossEntropyLoss()

In [9]:
def plot_cm(classes, cm):
    fig, ax = plt.subplots(figsize=(len(classes),len(classes)))  
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt="d")
    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels') 
    ax.set_title('Confusion Matrix') 
    ax.xaxis.set_ticklabels(classes,rotation=90)
    ax.yaxis.set_ticklabels(classes,rotation=0)
    return fig

In [10]:
def train(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    on_iteration=None,
):
    model = model.train()
    end = time.time()
    print("Start Training")
    avg_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        print(f"{i/len(train_loader) * 100 : 2.2f}%", end="\r")
        iteration_time = time.time()
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
        if on_iteration is not None:
            on_iteration(iteration=i+epoch*len(train_loader), loss=loss, y_pred=outputs, y_true=labels)     
    return avg_loss/len(train_loader)

In [11]:
def validate(val_loader, 
             model, 
             criterion,
             print_freq=1000):
    model = model.eval()
    y_pred, y_true = [], []
    avg_loss = 0
    for i, (inputs, labels) in enumerate(val_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        with torch.no_grad():
            outputs = model(inputs)
            avg_loss += criterion(outputs, labels).item()
        _, indices = torch.max(outputs.cpu(), 1)
        y_pred.append(indices)
        y_true.append(labels.cpu().clone())
    return avg_loss/len(val_loader), torch.cat(y_pred), torch.cat(y_true)

In [12]:
def compute_porn_detection(epoch, y_pred, y_true):
    ones = torch.ones(len(y_pred))
    p_pred = (y_pred == 1) | (y_pred == 3)
    p_truth = (y_true == 1) | (y_true == 3)
    logger.add_pr_curve("Eval/Prec_recall", p_truth, p_pred, epoch)

In [13]:
def validation_logs(epoch, loss_avg, y_pred, y_true):
    logger.add_scalar("Loss/Avg_Val", loss_avg, epoch)
    logger.add_scalar("Eval/Precision", (y_pred==y_true).sum()/float(len(y_pred)), epoch)
    compute_porn_detection(epoch, y_pred, y_true)
    cm = confusion_matrix(y_pred, y_true)
    cm_image = plot_cm(train_dataset.classes, cm)
    logger.add_figure("Eval/Confusion Matrix", cm_image, epoch)

In [14]:
def on_iteration_logs(iteration, loss, y_pred, y_true):
    l = loss.item()
    if iteration%50 == 0:
        logger.add_scalar("Loss/Train", l, iteration)
        print(
                f"{iteration}/{len(train_loader)} \t"
                f"Loss {l}"
            )

In [16]:
logger = SummaryWriter(TENSORBOARD_DIR)

In [ ]:
for i in range(10):
    val_loss, y_pred, y_true = validate(val_loader, model, criterion)
    validation_logs(i, val_loss, y_pred, y_true)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)

Start Training
0/1429 	Loss 1.645801305770874
50/1429 	Loss 0.8847590684890747
100/1429 	Loss 0.7891713380813599
150/1429 	Loss 0.7394140958786011
200/1429 	Loss 0.5289117693901062
250/1429 	Loss 0.4995679259300232
300/1429 	Loss 0.4629546105861664
350/1429 	Loss 0.5019838213920593
400/1429 	Loss 0.47636640071868896
450/1429 	Loss 0.48804357647895813
500/1429 	Loss 0.40514490008354187
550/1429 	Loss 0.39292454719543457
600/1429 	Loss 0.35952550172805786
650/1429 	Loss 0.4427937865257263
700/1429 	Loss 0.4801381826400757
750/1429 	Loss 0.4111124277114868
800/1429 	Loss 0.3206721246242523
850/1429 	Loss 0.41951197385787964
900/1429 	Loss 0.4607880115509033
950/1429 	Loss 0.3891167938709259
1000/1429 	Loss 0.3584074378013611
1050/1429 	Loss 0.38679301738739014
1100/1429 	Loss 0.36941391229629517
1150/1429 	Loss 0.3034724295139313
1200/1429 	Loss 0.39026010036468506
1250/1429 	Loss 0.29458874464035034
1300/1429 	Loss 0.3645346462726593
1350/1429 	Loss 0.4596782624721527
1400/1429 	Loss 0.4

In [ ]:
for i, child in enumerate(model.children()):
    if i > 6:
        for p in child.parameters():
            p.requires_grad=True

In [ ]:
TRAIN_BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)

In [ ]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.1)

In [ ]:
for i in range(10,20):
    val_loss, y_pred, y_true = validate(val_loader, model, criterion)
    validation_logs(i, val_loss, y_pred, y_true)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)